In [2]:
import pandas as pd

# ===== 데이터 로드 =====
file_path = "../data/customer_data.csv"
df = pd.read_csv(file_path)

In [3]:
# ===== 1. 결측치 확인 =====
print("=== 결측치 개수 확인 ===")
df.isna().sum()  # 각 컬럼별 결측치 개수 출력

=== 결측치 개수 확인 ===


CustomerID           0
Age                100
AnnualIncome       100
SpendingScore        0
PurchaseHistory      0
dtype: int64

In [3]:
# ===== 2. 결측치 처리 =====
# 'Age'와 'AnnualIncome' 컬럼의 결측값을 중앙값(median)으로 대체
df['Age'] = df['Age'].fillna(df['Age'].median())
df['AnnualIncome'] = df['AnnualIncome'].fillna(df['AnnualIncome'].median())

# 결측치 처리 후 다시 확인
print("\n=== 결측치 처리 후 확인 ===")
df.isna().sum() # 이제 결측치가 없어야 함


=== 결측치 처리 후 확인 ===


CustomerID         0
Age                0
AnnualIncome       0
SpendingScore      0
PurchaseHistory    0
dtype: int64

In [4]:
# ===== 3. 이상치 탐지 (IQR 방식) =====
# IQR(Interquartile Range)을 사용하여 이상치 경계값 구하는 함수
def get_iqr_bounds(series):
    """
    IQR(사분위 범위) 방법을 이용하여 이상치 경계값 반환
    - IQR = Q3(75%) - Q1(25%)
    - 이상치 기준: Q1 - 1.5*IQR 이하 또는 Q3 + 1.5*IQR 이상
    """
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR  # 하한선
    upper_bound = Q3 + 1.5 * IQR  # 상한선
    return lower_bound, upper_bound

# 각 컬럼별 이상치 경계값 계산
age_low, age_high = get_iqr_bounds(df['Age'])
income_low, income_high = get_iqr_bounds(df['AnnualIncome'])
score_low, score_high = get_iqr_bounds(df['SpendingScore'])

# 이상치 개수 확인
outliers_age = df[(df['Age'] < age_low) | (df['Age'] > age_high)]
outliers_income = df[(df['AnnualIncome'] < income_low) | (df['AnnualIncome'] > income_high)]
outliers_spending = df[(df['SpendingScore'] < score_low) | (df['SpendingScore'] > score_high)]

print("\n=== 이상치 탐지 결과 ===")
print(f"'Age' 이상치 개수: {len(outliers_age)}개")
print(f"'AnnualIncome' 이상치 개수: {len(outliers_income)}개")
print(f"'SpendingScore' 이상치 개수: {len(outliers_spending)}개")


=== 이상치 탐지 결과 ===
'Age' 이상치 개수: 1개
'AnnualIncome' 이상치 개수: 0개
'SpendingScore' 이상치 개수: 0개


In [5]:
# ===== 4. 이상치 처리 =====
# 이상치를 평균값으로 대체 (단, 평균값은 정상 범위 내 값으로 계산)
# 각 컬럼에서 이상치를 제외한 평균값 계산
age_mean = int(df[(df['Age'] >= age_low) & (df['Age'] <= age_high)]['Age'].mean())
income_mean = int(df[(df['AnnualIncome'] >= income_low) & (df['AnnualIncome'] <= income_high)]['AnnualIncome'].mean())
score_mean = int(df[(df['SpendingScore'] >= score_low) & (df['SpendingScore'] <= score_high)]['SpendingScore'].mean())

# 이상치를 평균값으로 대체
df.loc[(df['Age'] < age_low), 'Age'] = age_mean
df.loc[(df['Age'] > age_high), 'Age'] = age_mean

df.loc[(df['AnnualIncome'] < income_low), 'AnnualIncome'] = income_mean
df.loc[(df['AnnualIncome'] > income_high), 'AnnualIncome'] = income_mean

df.loc[(df['SpendingScore'] < score_low), 'SpendingScore'] = score_mean
df.loc[(df['SpendingScore'] > score_high), 'SpendingScore'] = score_mean

# 이상치 처리 후 다시 확인
print("\n=== 이상치 처리 후 확인 ===")
print(f"최소/최대 Age: {df['Age'].min()} ~ {df['Age'].max()}")
print(f"최소/최대 AnnualIncome: {df['AnnualIncome'].min()} ~ {df['AnnualIncome'].max()}")
print(f"최소/최대 SpendingScore: {df['SpendingScore'].min()} ~ {df['SpendingScore'].max()}")


=== 이상치 처리 후 확인 ===
최소/최대 Age: 18.0 ~ 69.0
최소/최대 AnnualIncome: 20.0 ~ 149.0
최소/최대 SpendingScore: 1 ~ 100


In [6]:
print(df)

     CustomerID   Age  AnnualIncome  SpendingScore  PurchaseHistory
0             1  47.0          63.0             19               12
1             2  31.0          73.0             67                7
2             3  43.0          85.5              1               17
3             4  33.0          66.0             71               15
4             5  22.0         127.0             61               11
..          ...   ...           ...            ...              ...
995         996  63.0         102.0             12               19
996         997  55.0          20.0             11                7
997         998  49.0         102.0             39                5
998         999  21.0         135.0             15               17
999        1000  29.0         108.0             30                3

[1000 rows x 5 columns]
